# Dataset
This will be the cell that we use to get random game boards and have Stockfish analyze

- The Chess engine will generate a random board state
- Stockfish will take that random board state and return an evaluation for white's position

In [ ]:
import chess
import chess.engine
import random
import numpy


# this function will create our x (board)
def random_board(max_moves=200):  # 200 should be enough moves to give enough data for AI
  board = chess.Board()
  depth = random.randrange(0, 200)

  for _ in range(depth):
    all_possible_moves = list(board.legal_moves)
    move = random.choice(all_possible_moves)
    board.push(move)
    if board.is_game_over():
      break

  return board

# this function will create our f(x) (score)
def stockfish(board, depth):
  with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
    result = sf.analyse(board, chess.engine.Limit(depth=depth))
    score = result['score'].white().score()
    return score

In [ ]:
board = random_board()
board

In [ ]:
print(stockfish(board, 10))

# Creating Data

In [ ]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

## This code generates and saves our data into a numpy file
- Generates random board
- Gets Stockfish eval
- Turns board into a 3d numpy array
- Saves eval and board array's to file

In [ ]:
boards = numpy.arange(89600000, dtype=numpy.int8).reshape(100000, 14, 8, 8)
evals = numpy.arange(100000, dtype=numpy.int32)

for i in range(100000):
    board = random_board()
    eval = stockfish(board, 10) 

    eval = eval if eval else 0
    
    boards[i] = split_dims(board)
    evals[i] = eval

numpy.savez('data.npz', b=boards, v=evals)

# Build Model

### Load In Data

In [1]:
import numpy

def get_dataset():
    data = numpy.load('data.npz')
    b, v = data['b'], data['v']
    v = numpy.asarray( v / abs(v).max() / 2 + 0.5, dtype=numpy.float32)

    return b, v

def get_test_dataset():
    data = numpy.load('testdata.npz')
    b, v = data['b'], data['v']
    v = numpy.asarray( v / abs(v).max() / 2 + 0.5, dtype=numpy.float32)

    return b, v


x_train, y_train = get_dataset()
y_train.resize(100000)

x_test, y_test = get_test_dataset()
y_test.resize(100000)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape=(14,8,8))

x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)

x = layers.Flatten()(x)
x = layers.Dense(32, tf.nn.sigmoid)(x)
outputs = layers.Dense(1, tf.nn.sigmoid)(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

2023-11-30 22:15:26.649565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 22:15:26.649618: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 22:15:26.649634: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 22:15:26.655444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 22:15:27.974760: I tensorflow/compiler/

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.MeanSquaredError())

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d (Conv2D)             (None, 14, 8, 32)         2336      
                                                                 
 batch_normalization (Batch  (None, 14, 8, 32)         128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 8, 32)         9248      
                                                                 
 batch_normalization_1 (Bat  (None, 14, 8, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 8, 64)         18496 

## Fit/Train Model

In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import datetime

name = "Chess-CNN-Model"
log_dir = "logs/" + name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

checkpoint_filepath = '/tmp/checkpoint/'
callbacks = [
    tb,

    ModelCheckpoint(
        filepath = checkpoint_filepath,
        save_weights_only=False,
        save_best_only = True,
    )
]

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10, verbose=1, validation_split=0.1, callbacks=callbacks)

model.save('models/model' + str(datetime.datetime.now()) + '.keras')

2023-11-30 22:15:28.871823: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2023-11-30 22:15:30.018241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-30 22:15:30.188083: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 22:15:30.813944: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f31018c5a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-30 22:15:30.813993: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2023-11-30 22:15:30.819117: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-30 22:15:30.885837: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3122/3125 [============================>.] - ETA: 0s - loss: 2.8562e-04INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


...

3125/3125 [==============================] - 23s 7ms/step - loss: 9.4162e-05 - val_loss: 9.7827e-05
Epoch 9/10
3121/3125 [============================>.] - ETA: 0s - loss: 9.2880e-05INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 22s 7ms/step - loss: 9.2871e-05 - val_loss: 9.2164e-05
Epoch 10/10
3117/3125 [============================>.] - ETA: 0s - loss: 9.1962e-05INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 22s 7ms/step - loss: 9.1907e-05 - val_loss: 9.1617e-05


In [6]:
print(model.evaluate(x_test, y_test, batch_size=32, verbose=1, callbacks=callbacks)*100)

3125/3125 [==============================] - 6s 2ms/step - loss: 9.1617e-05
0.009161710477201268
